<a href="https://colab.research.google.com/github/harishraisinghani/challenges-web3-json-rpc-scale/blob/main/Web3_data_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenges with reading Web3 data at scale
A Code Along to explore fetching wallet token balances using JSON-RPC point queries and the Covalent API

## Outline
1. What is JSON-RPC and how does it work
2. What is a wallet in Web3 (EVM context) and how do they work?
3. What is an ERC20 token? How do we create one?
4. What is the Covalent API?












In [ ]:
pip install web3

## What is JSON-RPC and how does it work?
JSON-RPC is a stateless, light-weight remote procedure call (RPC) protocol which defines several data structures (using JSON as the data format) and how they are processed over various transport methods such as web sockets, http and other message passing environments. See the full spec here: https://www.jsonrpc.org/specification

When it comes to reading blockchain data and sending transactions to a blockchain network, the JSON-RPC spec is implemented by every Ethereum and EVM-compatible (e.g. Polygon, BSC, Avalanche C-Chain, Fantom) client. This is why [Ethereum JSON-RPC methods](https://playground.open-rpc.org/?schemaUrl=https://raw.githubusercontent.com/ethereum/execution-apis/assembled-spec/openrpc.json&uiSchema%5BappBar%5D%5Bui:splitView%5D=false&uiSchema%5BappBar%5D%5Bui:input%5D=false&uiSchema%5BappBar%5D%5Bui:examplesDropdown%5D=false) such as `eth_getBalance` can be used the same way on all EVM-compatible blockchains.


![Example 1{caption=Figure 1: JSON-RPC data flow}](https://itzone.com.vn/wp-content/uploads/2020/03/a36ff732-6ff7-4076-a86b-1676d5af6c1e.png)


The JSON-RPC layer is a 'point-query' interface where data requests for a block or transaction is executed one object at a time so users are unable to query multiple objects at once and cannot batch export data(https://consensys.net/blog/cryptoeconomic-research/covalent-a-decentralized-blockchain-database-and-api/). This bottleneck is a primary reason why JSON-RPC doesn't scale when extracting on-chain data and why tools like the [Covalent API](https://www.covalenthq.com/platform) help address those shortfalls. 

To use the JSON-RPC layer and query the token balances of a wallet, we need a Web3 provider. There are lots of infrastructure providers out there such as Infura, Ankr, Chainstack, GetBlock and others. For the Ethereum Kovan testnet, we will use [Infura](https://infura.io/).


## Set up the Web3 provider
The following lines of code are used to set up our Web3 provider using the Infura API:

In [ ]:
from web3 import Web3

kovan_provider = 'https://kovan.infura.io/v3/9aa3d95b3bc440fa88ea12eaa4456161'

w3_kovan = Web3(Web3.HTTPProvider(kovan_provider))

# Check that it is connected
w3_kovan.isConnected()

True

Now let us test a few simple [JSON-RPC calls](https://playground.open-rpc.org/?schemaUrl=https://raw.githubusercontent.com/ethereum/execution-apis/assembled-spec/openrpc.json&uiSchema%5BappBar%5D%5Bui:splitView%5D=false&uiSchema%5BappBar%5D%5Bui:input%5D=false&uiSchema%5BappBar%5D%5Bui:examplesDropdown%5D=false). See how they are mapped in `Web3.py` [here](https://web3py.readthedocs.io/en/stable/web3.eth.html#methods).

*   `eth_blockNumber` -> `web3.eth.get_block_number()`
*   `eth_gasPrice` -> `web3.eth.gas_price`



In [ ]:
w3_kovan.eth.get_block_number()

32165055

In [ ]:
w3_kovan.eth.gas_price # Current gas price in Wei

2500000007

## Create a Web3 wallet
Wallets are fundamental in Web3 as they serve as a digital identity to access applications. Moreover, they provide *access* to digital assets such as fungible and non-fungible tokens (NFTs). A key point to remember is that a wallet does not actually contain any ether or tokens. 

At its core, a wallet consists of three components:
1. **A public key** which serves as an address to send and receive transactions. Think of a public key as a publicly available bank account that someone can send funds to. The public key is derived from a private key.

2. **A private key** (which you should always keep private) which is a 256 character binary (64 digit hexadecimal / 32 bytes) code. The private key is an astronomically large number for good reason and while the public key is derived from it, the reverse is practically impossible (https://www.gemini.com/cryptopedia/public-private-keys-cryptography)

3. **A mnenomic (seed phrase)** of usually 12 or 24 words  that can uniquey recreate the private key. It should also always be kept private. See the [BIP-39](https://www.blockplate.com/blogs/blockplate/list-of-bip39-wallets-mnemonic-seed) standard. 

We are going to use the [`eth_account`](https://eth-account.readthedocs.io/en/stable/eth_account.html#module-eth_account.account) Python library (which was automatically installed with Web3.py) to create local private and public keys. 

**NOTE: EVERYTHING WE DO HERE IS FOR DEMO PURPOSES SO PLEASE DO NOT USE ANY WALLETS HERE AS YOUR PRODUCTION WALLET!**

In [ ]:
from eth_account import Account

Account.enable_unaudited_hdwallet_features()
acct, mnemonic = Account.create_with_mnemonic()

wallet_address = acct.address
private_key = acct.privateKey

print(f'My wallet address: {wallet_address}')
print(f'My wallet mnemonic that I should never share: "{mnemonic}"')

My wallet address: 0x8D1cb70700eE61e8dbda42eaCa5fB2f07d50262a
My wallet mnemonic that I should never share: "clever final diesel much attend harsh banner much barrel file drastic chunk"


So here we now have our public wallet address and our mnemonic that we should never share. We will use this mnemonic to set up our newly created wallet with the MetaMask browser wallet. 

**NOTE: IF YOU ALREADY HAVE METAMASK INSTALLED, MAKE SURE TO STORE YOUR EXISTING MNEMONIC SOMEWHERE SECURE BEFORE MOVING TO THE NEXT STEP OTHERWISE YOU MIGHT LOSE ACCESS TO YOUR WALLET.**

### Bonus:
Another way we can create our Web3 wallet out of thin air is by randomly creating our 64 hex / 32 bytes private key and then deriving the public key as follows using Python's [`secrets`](https://docs.python.org/3/library/secrets.html) library:


In [ ]:
import secrets
priv = secrets.token_hex(32) # 32 random bytes with each byte converted to two hex digits
other_private_key = "0x" + priv
account = Account.from_key(other_private_key)
other_wallet_address = account.address
print("Wallet address is:", other_wallet_address)
print("Private key is:", other_private_key)

Wallet address is: 0x0A1073a1CB8a0604525080762D142b35E4E000A8
Private key is: 0xbdc105f3d6897a248e5ec19aef7020b66918f80fb49f53fe017c2469421ec710


## MetaMask reset wallet

If you have MetaMask installed, log out and then select *Forgot password*. 

You will then be prompted with page to reset your wallet using your Secret Recovery Phrase. Paste your entire mnemonic from above into the first field. 

![Reset MetaMask Wallet{caption=Figure 2: MetaMask reset wallet}](https://metamask.zendesk.com/hc/article_attachments/5998778572315/SRP_entry.png)

Set you password and you should now have imported your wallet generated above into your MetaMask!

Now that you have connected your wallet, go to https://faucets.chain.link/ and drip yourself some test ETH and LINK tokens for the Ethereum Kovan test network. 

### Check your wallet balance

Now that you have 0.1 ETH (on Kovan test network) accessible by your wallet, let us check the balance



In [ ]:
w3_kovan.eth.get_balance(wallet_address)

100000000000000000

You also have 20 test LINK tokens. How do we check the balance of that using JSON-RPC calls?

We need to have the deployed LINK token contract address on Kovan as well as its Application Binary Interface (ABI). The ABI is the human-readable representation of a smart contract and defines its methods and structures in JSON format.  

This process of fetching first the token contract address and its ABI holds true for any token that you want to get the balance for. 

*   Chainlink token on Kovan details: https://docs.chain.link/docs/link-token-contracts/#kovan
*   Chainlink token ABI: https://etherscan.io/address/0x514910771af9ca656af840dff83e8264ecf986ca#code

We can now create an instance of the LINK token contract and call its methods:





In [ ]:
kovan_link_address = '0xa36085F69e2889c224210F603D836748e7dC0088'

In [ ]:
abi = [{"constant":True,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":False,"stateMutability":"view","type":"function"},{"constant":False,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":False,"stateMutability":"nonpayable","type":"function"},{"constant":True,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":False,"stateMutability":"view","type":"function"},{"constant":False,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":False,"stateMutability":"nonpayable","type":"function"},{"constant":True,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":False,"stateMutability":"view","type":"function"},{"constant":False,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"},{"name":"_data","type":"bytes"}],"name":"transferAndCall","outputs":[{"name":"success","type":"bool"}],"payable":False,"stateMutability":"nonpayable","type":"function"},{"constant":False,"inputs":[{"name":"_spender","type":"address"},{"name":"_subtractedValue","type":"uint256"}],"name":"decreaseApproval","outputs":[{"name":"success","type":"bool"}],"payable":False,"stateMutability":"nonpayable","type":"function"},{"constant":True,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"payable":False,"stateMutability":"view","type":"function"},{"constant":True,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":False,"stateMutability":"view","type":"function"},{"constant":False,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[{"name":"success","type":"bool"}],"payable":False,"stateMutability":"nonpayable","type":"function"},{"constant":False,"inputs":[{"name":"_spender","type":"address"},{"name":"_addedValue","type":"uint256"}],"name":"increaseApproval","outputs":[{"name":"success","type":"bool"}],"payable":False,"stateMutability":"nonpayable","type":"function"},{"constant":True,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"remaining","type":"uint256"}],"payable":False,"stateMutability":"view","type":"function"},{"inputs":[],"payable":False,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":False,"inputs":[{"indexed":True,"name":"from","type":"address"},{"indexed":True,"name":"to","type":"address"},{"indexed":False,"name":"value","type":"uint256"},{"indexed":False,"name":"data","type":"bytes"}],"name":"Transfer","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"name":"owner","type":"address"},{"indexed":True,"name":"spender","type":"address"},{"indexed":False,"name":"value","type":"uint256"}],"name":"Approval","type":"event"}]

In [ ]:
link_token = w3_kovan.eth.contract(address=kovan_link_address, abi=abi)

In [ ]:
symbol = link_token.functions.symbol().call()
symbol

'LINK'

In [ ]:
balance = link_token.functions.balanceOf(wallet_address).call()
balance

20000000000000000000

So now we can see pretty quickly the limitations of making JSON-RPC calls to fetch token balances for a wallet address. This is where tools like the Covalent API address those limitations. 

## The Covalent API

[Covalent](https://www.covalenthq.com) provides a unified API to bring full transparency and visibility to assets across all blockchains.

To get started, sign up for an [API Key](https://www.covalenthq.com/platform). 


In [ ]:
API_KEY = 'ckey_6bf60a7bf22d4a309dbe74f3c5c'

Then we use Python's Requests library to make a standard HTTP GET request to the [`Get token balances for address`](https://www.covalenthq.com/docs/api/#/0/Get%20token%20balances%20for%20address/USD/42) API endpoint.

In [ ]:
import requests

chain_id = 42 # for Kovan

url = f'https://api.covalenthq.com/v1/{chain_id}/address/{wallet_address}/balances_v2/?key={API_KEY}'

r = requests.get(url)
data = r.json()

data

{'data': {'address': '0x8d1cb70700ee61e8dbda42eaca5fb2f07d50262a',
  'chain_id': 42,
  'items': [{'balance': '100000000000000000',
    'balance_24h': '0',
    'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee',
    'contract_decimals': 18,
    'contract_name': 'Ether',
    'contract_ticker_symbol': 'ETH',
    'last_transferred_at': None,
    'logo_url': 'https://www.covalenthq.com/static/images/icons/display-icons/ethereum-eth-logo.png',
    'nft_data': None,
    'quote': 117.34874,
    'quote_24h': None,
    'quote_rate': 1173.4874,
    'quote_rate_24h': None,
    'supports_erc': None,
    'type': 'cryptocurrency'},
   {'balance': '20000000000000000000',
    'balance_24h': '0',
    'contract_address': '0xa36085f69e2889c224210f603d836748e7dc0088',
    'contract_decimals': 18,
    'contract_name': 'ChainLink Token',
    'contract_ticker_symbol': 'LINK',
    'last_transferred_at': '2022-06-14T04:33:52Z',
    'logo_url': 'https://logos.covalenthq.com/tokens/1/0xa36085f69e288

We see that the data structure of the API response contains a list of `'items'` that we can iterate through to display the balance (in Wei) of EACH token that this wallet has access too. 

In [ ]:
tokens = data['data']['items']

for token in tokens:
  print(f'{token["contract_ticker_symbol"]}:{float(token["balance"])/10**token["contract_decimals"]}')

ETH:0.1
LINK:20.0


And that's all you need to do to fetch all the ERC20 tokens using the Covalent API!